In [1]:
import csv
import re

input_file = "Bank_Transaction.csv"
output_file = "Cleaned_Bank_Transaction.csv"

with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    writer.writerow(['txn_date', 'description', 'debit', 'credit', 'balance'])  # optional, i prefer to rename my header

    # Merge Multi-line Descriptions
    #Sometimes, a single transaction is split across multiple lines : the date and numbers are on one row, but the description continues on the next row.
    # MySQL cannot read this properly
    #Open the csv file using notebook to understand this
    buffer = [] #temporary storage to hold each transaction before saving it in one line
    for row in reader: #Loop through each row (line) in the CSV file
        if re.match(r'^\d{1,2}/\d{1,2}/\d{4}$', row[0].strip()) or re.match(r'^\d{4}-\d{2}-\d{2}$', row[0].strip()):
            # This checks: Does the first cell look like a date? if yes then new transaction start if not then just description
            if buffer:
                writer.writerow(buffer)
            buffer = row
        else:
            # If it’s a continuation of description
            if buffer:
                buffer[1] += ' ' + ' '.join(row).strip()

    # If buffer already has a full row stored, write it to the output file
    if buffer:
        writer.writerow(buffer)

# Clean numeric values (remove commas)
import pandas as pd

df = pd.read_csv(output_file)

for col in ['debit', 'credit', 'balance']:
    df[col] = (
        df[col].astype(str)
        .str.replace(',', '', regex=False)
        .str.strip()
        .replace('', None)
        .astype(float)
    )

df.to_csv(output_file, index=False)
